In [1]:
!pip install dist/alquimia_fair_forge-0.0.1.tar.gz -q


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from helpers.retriever import LocalRetriever
from fair_forge.metrics import Bias
from pydantic import SecretStr
import os

/Users/alexfiorenza/Documents/software_development/projects/alquimia/fair-forge/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from getpass import getpass
guardian_api_key = SecretStr(getpass("Please enter your Guardian LLM API key: "))

In [4]:
ELASTIC_URL = os.environ.get('ELASTIC_URL')
ELASTIC_AUTH = [os.environ.get('ELASTIC_AUTH_USER'), os.environ.get('ELASTIC_AUTH_PASSWORD')]
dataset = os.environ.get("dataset", "asb")
bias_index = f"{dataset}-bias"
guardian_temperature = 0.5
max_tokens = 5

In [5]:
## GRANITE
GUARDIAN_URL = os.environ.get("GUARDIAN_URL","https://runtime.apps.hostmydemo.online")
GUARDIAN_MODEL_NAME = os.environ.get("GUARDIAN_MODEL_NAME","ibm-granite/granite-guardian-3.1-2b")
GUARDIAN_API_KEY = guardian_api_key

In [5]:
## LLAMA GUARD
GUARDIAN_URL = "https://api.groq.com/openai"
GUARDIAN_MODEL_NAME = "meta-llama/Llama-Guard-4-12B"
hf_token = SecretStr(getpass("Please enter your Hugging Face token: "))
os.environ["HF_TOKEN"] = hf_token.get_secret_value()
GUARDIAN_API_KEY = guardian_api_key

In [6]:
from fair_forge.guardians import IBMGranite,LLamaGuard
from fair_forge.guardians.llms.providers import OpenAIGuardianProvider
from fair_forge.schemas import GuardianLLMConfig

In [9]:
metrics= Bias.run(
    LocalRetriever,
    guardian = LLamaGuard,
    confidence_level= 0.90,
    config = GuardianLLMConfig(
        model= GUARDIAN_MODEL_NAME,
        api_key= GUARDIAN_API_KEY.get_secret_value(),
        url=GUARDIAN_URL,
        temperature=guardian_temperature,
        provider=OpenAIGuardianProvider
    ),
    verbose=True
)

2025-05-23 12:05:28,129 - fair_forge - INFO - Loaded dataset with 1 batches
2025-05-23 12:05:28,841 - fair_forge - INFO - Starting to process dataset
2025-05-23 12:05:28,841 - fair_forge - INFO - Processing batch 1/1
2025-05-23 12:05:28,841 - fair_forge - INFO - Session ID: 123, Assistant ID: my_assistant
2025-05-23 12:05:28,842 - fair_forge - INFO - QA ID: 123
2025-05-23 12:05:31,812 - fair_forge - INFO - QA ID: 124
2025-05-23 12:05:34,258 - fair_forge - INFO - QA ID: 125
2025-05-23 12:05:36,542 - fair_forge - INFO - QA ID: 126
2025-05-23 12:05:39,289 - fair_forge - INFO - QA ID: 127
2025-05-23 12:05:41,849 - fair_forge - INFO - QA ID: 128
2025-05-23 12:05:44,377 - fair_forge - INFO - QA ID: 129
2025-05-23 12:05:46,812 - fair_forge - INFO - QA ID: 130
2025-05-23 12:05:49,534 - fair_forge - INFO - [gender]: Clopper-Pearson Confidence Interval: [0.34462318685401877 - 0.625 - 0.8531450676486562]
2025-05-23 12:05:49,536 - fair_forge - INFO - [race]: Clopper-Pearson Confidence Interval: [0

In [ ]:
es = Elasticsearch(
    ELASTIC_URL,
    basic_auth=tuple(ELASTIC_AUTH),
)

In [ ]:
def recreate_index(index_name: str, mapping: dict):
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"Index '{index_name}' deleted.")
    es.indices.create(index=index_name, body=mapping)
    print(f"Index '{index_name}' created.")

In [ ]:
def flatten_guard_metrics(metrics):
    flattened = []
    for metric in metrics:
        for risk in metric.risks:
            flattened.append(
                {
                    "session_id": metric.session_id,
                    "qa_id": metric.qa_id,
                    "assistant_id": metric.assistant_id,
                    'bias_guard_is_risk': risk.is_risk,
                    'bias_guard_type': risk.risk_type,
                    'bias_guard_probability': risk.probability
                }
            )
    return flattened
flattened = flatten_guard_metrics(metrics)

In [ ]:
mapping_bias = {
  "mappings": {
    "properties": {
      "session_id": {"type": "keyword"},
      "bias_guard_is_risk": {"type": "boolean"},
      "bias_guard_type": {"type": "text"},
      "bias_guard_probability": {"type": "float"},
      "assistant_id": {"type": "keyword"},
      "qa_id": {"type": "keyword"},
    }
  }
}

In [ ]:
recreate_index(bias_index, mapping_bias)

In [ ]:
docs = []
for flatten in flattened:
    docs.append({
            "_index": bias_index,
            "_source": flatten
    })

helpers.bulk(es, docs)
print(f"Indexed {len(docs)} documents.")